## Execute SQL Queries against SQLite DB
The notebook executes the different versions of SQL statements against the SQLite database and check if each statement is executable.
* Ground-truth SQL
* GPT4 SQL responses
* GPT4 improved SQL responses

In [1]:
DB_PATH = "../database/synpuf.db"

# QA_DATA_PATH = "s3://aws-gaiic-merck-text2sql/data/synpuf_query_combined.csv"
QA_DATA_PATH = "s3://aws-gaiic-merck-text2sql/data/synpuf_query_combined_20231116.csv"

#Redshift params
HOST = "redshift-synpuf-db-cluster-1.ckbiikj73zqo.us-east-1.redshift.amazonaws.com"
PORT = 5439
DATABASE = 'dev'#'dev' #'rwdex_raw_synpuf'
USER = "awsuser"
PASSWORD = "Merck123"

In [9]:
import sqlite3
import pandas as pd
import redshift_connector
from IPython.display import display, HTML

pd.set_option('display.max_colwidth', None)

import warnings
warnings.filterwarnings('ignore')

In [17]:
#Read qa csv data
df_qa = pd.read_csv(QA_DATA_PATH)
print(df_qa.shape)
df_qa.head()

(40, 14)


qa_id  \
0    t1   
1    t2   
2    t3   
3    t4   
4    t5   

                                                                                                                                                                                                                                                                 query  \
0                                                                                                                                                                                          select count(distinct desynpuf_id) from rwdex_raw_synpuf.outpatient_claims;   
1                                                                                                                                                                                           select count(distinct desynpuf_id) from rwdex_raw_synpuf.inpatient_claims;   
2                                                                                                                                           select distinct desynpuf_id from rwdex_raw_synpuf.inpatient_claims where clm_from_dt between '2005-01-01' and '2006-01-01'   
3                                                                                                                                          select distinct desynpuf_id from rwdex_raw_synpuf.outpatient_claims where clm_from_dt between '2005-01-01' and '2006-01-02'   
4  select distinct desynpuf_id from rwdex_raw_synpuf.outpatient_claims \nwhere clm_from_dt between '2015-01-01' and '2016-01-01' \nunion select distinct desynpuf_id from rwdex_raw_synpuf.inpatient_claims\nwhere clm_admsn_dt between '2015-01-01' and '2016-01-01'    

                                                                                                                                                                                                                                                       query_corrected  \
0                                                                                                                                                                                          select count(distinct desynpuf_id) from rwdex_raw_synpuf.outpatient_claims;   
1                                                                                                                                                                                           select count(distinct desynpuf_id) from rwdex_raw_synpuf.inpatient_claims;   
2                                                                                                                                           select distinct desynpuf_id from rwdex_raw_synpuf.inpatient_claims where clm_from_dt between '2005-01-01' and '2006-01-01'   
3                                                                                                                                          select distinct desynpuf_id from rwdex_raw_synpuf.outpatient_claims where clm_from_dt between '2005-01-01' and '2006-01-02'   
4  select distinct desynpuf_id from rwdex_raw_synpuf.outpatient_claims \nwhere clm_from_dt between '2015-01-01' and '2016-01-01' \nunion select distinct desynpuf_id from rwdex_raw_synpuf.inpatient_claims\nwhere clm_admsn_dt between '2015-01-01' and '2016-01-01'    

                                                                                                                                                                                                                                             query_corrected_with_list  \
0                                                                                                                                                                                          select count(distinct desynpuf_id) from rwdex_raw_synpuf.outpatient_claims;   
1                                                                                                                                                                                           select count(distinct

In [18]:
#Group based on difficulty level
print('Total Q/A Pairs:', len(df_qa))
df_qa.difficulty_level.value_counts()

Total Q/A Pairs: 40


3    16
4    10
2     7
5     5
1     2
Name: difficulty_level, dtype: int64

In [12]:
#create db connection
con_sqlite = sqlite3.connect(DB_PATH)

#Using redshift connector
con_redshift = redshift_connector.connect(
    host=HOST,
    database=DATABASE,
    port=PORT,
    user=USER,
    password=PASSWORD
)

In [30]:
def run_df_row_query(row0, column, con, con_type='redshift'):
    """Run SQL query from the given dataframe row's column."""
    row = row0.copy()
    
    sql = row[column]
    if column == 'gpt4_revised_response':
        if pd.isna(row['gpt4_revised_response']):
            sql = row['gpt4_response']
            
    if isinstance(sql, str) and (con_type!='redshift'):
        sql = sql.replace('rwdex_raw_synpuf.', '')
    
    print(f"QA ID: {row['qa_id']}")
    print('-'*30)

    display(HTML(f"<font color=blue>QUESTION: {row['question']}</font>"))
    print('-'*30)

    try:
        df = pd.read_sql(sql, con)
        row[f'{column}_executed'] = True
        display(HTML(f"<font color=green>SQL: {sql}</font>"))
        print('-'*30)

        print('Total Rows:', len(df))
        print('-'*30)
        print('Sample data:')
        display(df.head())
    
    except Exception as exp:
        row[f'{column}_executed'] = False
        display(HTML(f"<font color=red>SQL: {sql}</font>"))
        print('-'*30)
        display(HTML(f"<i><u>Error Message:</u> {exp}</i>"))
        
    print('*'*60)
    return row

def run_original_and_final_query_row(row0, con, con_type):
    """Run the updated version of SQL ground-truth query from the given dataframe row's column."""
    row = row0.copy()
    
    sql_original = row['query']
    sql_final = row['query_corrected_with_list']
    
    qn_original = row['question']
    qn_final = row['question_with_list']
    
    if isinstance(sql_original, str)  and (con_type!='redshift'):
        sql_original = sql_original.replace('rwdex_raw_synpuf.', '')

    if isinstance(sql_final, str) and (con_type!='redshift'):
        sql_final = sql_final.replace('rwdex_raw_synpuf.', '')

    print(f"QA ID: {row['qa_id']}")
    print('-'*30)

    display(HTML(f"<font color=blue>QUESTION[original]: {qn_original}</font>"))
    print('-'*30)
    display(HTML(f"<font color=blue>QUESTION[final]: {qn_final}</font>"))
    print('-'*30)

    try:
        df = pd.read_sql(sql_original, con)
        display(HTML(f"<font color=green>SQL[original]: {sql_original}</font>"))
        print('-'*30)
    except Exception as exp:
        row[f'{column}_executed'] = False
        display(HTML(f"<font color=red>SQL[original]: {sql_original}</font>"))
        print('-'*30)
        display(HTML(f"<i><u>Error Message:</u> {exp}</i>"))
        print('-'*30)

    try:
        df = pd.read_sql(sql_final, con)
        display(HTML(f"<font color=green>SQL[final]: {sql_final}</font>"))
        print('-'*30)
    except Exception as exp:
        row[f'{column}_executed'] = False
        display(HTML(f"<font color=red>SQL[final]: {sql_final}</font>"))
        print('-'*30)
        display(HTML(f"<i><u>Error Message:</u> {exp}</i>"))
        
    print('*'*60)

In [19]:
column = 'query'
con_type='redshift'
print(f'Analysis for {column}\n{"="*50}')
df_qa = df_qa.apply(run_df_row_query, args=(column, con_redshift, con_type), axis=1)

Analysis for query
QA ID: t1
------------------------------


------------------------------


------------------------------
Total Rows: 1
------------------------------
Sample data:


count
0  85272

************************************************************
QA ID: t2
------------------------------


...

------------------------------


************************************************************
QA ID: t40
------------------------------


------------------------------


------------------------------


************************************************************


In [20]:
df_qa[f'{column}_executed'].value_counts()

False    22
True     18
Name: query_executed, dtype: int64

In [21]:
column = 'query_corrected_with_list'
con_type='redshift'
print(f'Analysis for {column}\n{"="*50}')
df_qa = df_qa.apply(run_df_row_query, args=(column, con_redshift, con_type), axis=1)

Analysis for query_corrected_with_list
QA ID: t1
------------------------------


------------------------------


------------------------------
Total Rows: 1
------------------------------
Sample data:


count
0  85272

************************************************************
QA ID: t2
------------------------------


...

------------------------------


************************************************************
QA ID: t40
------------------------------


------------------------------


------------------------------


************************************************************


In [22]:
df_qa[f'{column}_executed'].value_counts()

True     30
False    10
Name: query_corrected_with_list_executed, dtype: int64

In [23]:
column = 'query_corrected_sqlite_with_list'
con_type='sqlite'
print(f'Analysis for {column}\n{"="*50}')
df_qa = df_qa.apply(run_df_row_query, args=(column, con_sqlite, con_type), axis=1)

Analysis for query_corrected_sqlite_with_list
QA ID: t1
------------------------------


------------------------------


------------------------------
Total Rows: 1
------------------------------
Sample data:


count(distinct desynpuf_id)
0                        85272

************************************************************
QA ID: t2
------------------------------


...

Empty DataFrame
Columns: [DESYNPUF_ID, min(clm_from_dt), max(clm_thru_dt), duration]
Index: []

************************************************************
QA ID: t40
------------------------------


------------------------------


------------------------------


************************************************************


In [24]:
df_qa[f'{column}_executed'].value_counts()

True     36
False     4
Name: query_corrected_sqlite_with_list_executed, dtype: int64

In [ ]:
# column = 'gpt4_response'
# print(f'Analysis for {column}\n{"="*50}')
# df_qa = df_qa.apply(run_df_row_query, args=(column, con), axis=1)

In [ ]:
# df_qa[f'{column}_executed'].value_counts()

In [ ]:
# column = 'gpt4_revised_response'
# print(f'Analysis for {column}\n{"="*50}')
# df_qa = df_qa.apply(run_df_row_query, args=(column, con), axis=1)

In [ ]:
# df_qa[f'{column}_executed'].value_counts()

## Compare the Different Versions of SQL Queries

In [31]:
print('Comparing original and final SQL queries....')
con_type='redshift'
_ = df_qa.apply(run_original_and_final_query_row, args=(con_redshift, con_type), axis=1)

Comparing original and final SQL queries....
QA ID: t1
------------------------------


------------------------------


------------------------------


------------------------------


------------------------------
************************************************************
QA ID: t2
------------------------------


...

------------------------------


------------------------------


------------------------------


------------------------------


************************************************************


In [32]:
con_sqlite.close()
con_redshift.close()

In [33]:
#Save to file

In [34]:
OUTPUT_PATH = QA_DATA_PATH.replace('.csv', '_executed.csv')
df_qa.to_csv(OUTPUT_PATH, index=False)